In [9]:
#Install sodapy if not installed already
#!pip install sodapy

In [4]:
from sodapy import Socrata
import pandas as pd

In [5]:
client = Socrata("data.sfgov.org", None)

In [6]:
results = client.get("wg3w-h783", limit=2000)

In [7]:
results_df = pd.DataFrame.from_records(results)

In [8]:
results_df

,:@computed_region_26cr_cadq,:@computed_region_2dwj_jsy4,:@computed_region_6pnf_4xz7,:@computed_region_6qbp_sg9q,:@computed_region_ajp5_b2md,:@computed_region_h4ep_8xdi,:@computed_region_jg9y_a9du,:@computed_region_nqbw_i6c3,:@computed_region_qgnn_b9vv,:@computed_region_y6ts_4iup,...,latitude,longitude,point,police_district,report_datetime,report_type_code,report_type_description,resolution,row_id,supervisor_district
0,7,NaN,1,39,35,NaN,NaN,NaN,10,NaN,...,37.76256939715695,-122.49962745519909,"{'latitude': '37.76256939715695', 'longitude':...",Taraval,2019-06-12T20:27:00.000,II,Initial,Open or Active,81097515200,4
1,10,NaN,2,32,34,1,NaN,1,1,NaN,...,37.7805353858225,-122.40816079455212,"{'latitude': '37.7805353858225', 'longitude': ...",Southern,2019-06-22T08:05:00.000,II,Initial,Open or Active,81465564020,6
2,9,NaN,2,88,1,NaN,NaN,NaN,2,NaN,...,37.72159985216247,-122.39074534279013,"{'latitude': '37.72159985216247', 'longitude':...",Bayview,2019-06-03T16:16:00.000,IS,Initial Supplement,Open or Active,80769875000,10
3,3,18,2,104,6,NaN,NaN,NaN,6,NaN,...,37.794859532228344,-122.40487561154785,"{'latitude': '37.794859532228344', 'longitude'...",Central,2018-11-16T16:34:00.000,IS,Initial Supplement,Cite or Arrest Adult,73857915041,3
4,6,NaN,1,15,13,NaN,NaN,NaN,4,NaN,...,37.79771621229674,-122.43055896140594,"{'latitude': '37.79771621229674', 'longitude':...",Northern,2019-05-27T02:55:00.000,II,Initial,Open or Active,80509204134,2
5,10,NaN,1,32,8,NaN,NaN,NaN,1,NaN,...,37.78582921318811,-122.40148983641853,"{'latitude': '37.78582921318811', 'longitude':...",Central,2018-11-15T11:28:00.000,II,Initial,Open or Active,73795805151,6
6,6,NaN,1,13,18,NaN,NaN,NaN,8,NaN,...,37.782488031626265,-122.4458205129012,"{'latitude': '37.782488031626265', 'longitude'...",Richmond,2019-08-15T17:23:00.000,II,Initial,Open or Active,83489506244,2
7,10,NaN,2,32,34,1,NaN,1,5,NaN,...,37.780005867994824,-122.41193341005338,"{'latitude': '37.780005867994824', 'longitude'...",Tenderloin,2019-08-07T07:47:00.000,II,Initial,Open or Active,83165364085,6
8,11,NaN,2,97,39,NaN,NaN,NaN,4,NaN,...,37.78080176962032,-122.43726005920409,"{'latitude': '37.78080176962032', 'longitude':...",Northern,2018-11-15T15:56:00.000,IS,Initial Supplement,Cite or Arrest Adult,73829775025,5
9,6,NaN,1,99,32,NaN,NaN,NaN,6,NaN,...,37.806780111468534,-122.4195772441978,"{'latitude': '37.806780111468534', 'longitude'...",Central,2019-06-29T16:52:00.000,II,Initial,Cite or Arrest Adult,81679765010,2
